# Tweets preprocessing

install libraries

In [ ]:
# Data preprocessing for Crisis dataset v1.1

# This notebook is created in Google Colab, please change the paths to your file.
# Every preprocessing method is seperate. You can choose the ones that you need. 
# Write me a note if something goes wrong or you need some new preprocessing methods.

# Update from v1.0:
# Include crisis_dataset_1
# Include evaluation

# Enjoy!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 14.9 MB/s 
     |████████████████████████████████| 106 kB 63.8 MB/s 


In [ ]:
import pandas as pd
import unicodedata
import re
import contractions
from pathlib import Path
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

Load many datasets

In [ ]:
def split_dataset(tweets_df, seed = 100, TRAIN = 0.8, DEV = 0.1, TEST = 0.1):
    # Define the partition of train, dev and test set, make sure they sum up to 1
    PT_TRAIN = TRAIN
    PT_DEV = DEV
    PT_TEST = TEST

    # Define train, dev and test set
    tweets_df.loc[: tweets_df.shape[0] * PT_TRAIN, 'partition'] = 'train'
    tweets_df.loc[tweets_df.shape[0] * PT_TRAIN : tweets_df.shape[0] * (PT_TRAIN + PT_DEV), 'partition'] = 'dev'
    tweets_df.loc[tweets_df.shape[0] * (PT_TRAIN + PT_DEV) : tweets_df.shape[0] - 1, 'partition'] = 'test'

    # Shuffle
    final_df = tweets_df.sample(frac=1, random_state = seed).reset_index(drop=True)

    return final_df

In [ ]:
def load_documents(dataset_dir: str, dataset_text_col: str):
   # This method comes from Berk's work: src/utils.py
   # Input:
   # dataset_dir: The folder where the datasets are stored.
   # dataset_text_col: The name of the column that contains tweets.
   # (For dataset 1: tweets_text, for dataset 12: text)

   # Output:
   # One dataframe containing all the data

   dataset_data_paths = sorted([path for path in Path(dataset_dir).iterdir() if path.suffix in {'.csv', '.tsv'}])
   
   dfs = []
   labels_as_filenames = []
   topic_num = 0
   for data_path in dataset_data_paths:
      csv_delimiter = '\t' if data_path.suffix == '.tsv' else ','
      df = pd.read_csv(data_path, delimiter=csv_delimiter)
      df = df[[dataset_text_col]]
      splitted_df = split_dataset(df)
      splitted_df.loc[:, 'Topics'] = f'{topic_num}'
      topic_num += 1
      # Why do we need this?
      labels_as_filenames.extend([Path(data_path).stem] * len(df))
      dfs.append(splitted_df)
   
   merged_df = pd.concat(dfs, axis=0)
   merged_df.rename(columns={f'{dataset_text_col}':'Tweets'},inplace=True)

   return merged_df
   #assert False

In [ ]:
# Select the right dataset

# Crisis dataset No.1
#dataset_dir = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_1_original'
#save_dir = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_1'
#tweets_df = load_documents(dataset_dir, 'tweet_text')
#NUM_TOPICS = 11

# Crisis dataset No.12
dataset_dir = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12_original'
save_dir = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12'
tweets_df = load_documents(dataset_dir, 'text')
NUM_TOPICS = 4

In [ ]:
tweets_df

,Tweets,partition,Topics
0,Bay Area niggas jus felt that lil earthquake l...,train,0
1,jswartz Yea! Welcome to earthquake twitter!,train,0
2,"I think that was a light earthquake, but I won...",train,0
3,Did y all feel that earthquake?,train,0
4,Another freaking #earthquake ... what gives?! ...,train,0
...,...,...,...
1995,Malcolm Turnbull has praised technology for an...,test,3
1996,#NEWS #Cyclone Cyclone Debbie: What will happe...,test,3
1997,Watch Tropical Cyclone Debbie intensify from s...,test,3
1998,Cyclone dumps bull shark in middle of road htt...,train,3


In [ ]:
'''
# Former method

path1 = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/earthquakes_eyewitness_crowdflower_2000.tsv'
path2 = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/floods_eyewitness_crowdflower_2000.tsv'
path3 = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/forestfires_eyewitness_crowdflower_2000.tsv'
path4 = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/hurricanes_eyewitness_crowdflower_2000.tsv'

tweets_df1=pd.read_csv(path1, sep="\t")
tweets_df2=pd.read_csv(path2, sep="\t")
tweets_df3=pd.read_csv(path3, sep="\t")
tweets_df4=pd.read_csv(path4, sep="\t")

tweets_df1 = tweets_df1[['text']]
tweets_df2 = tweets_df2[['text']]
tweets_df3 = tweets_df3[['text']]
tweets_df4 = tweets_df4[['text']]

tweets_df1.rename(columns={'text':'Tweets'},inplace=True)
tweets_df2.rename(columns={'text':'Tweets'},inplace=True)
tweets_df3.rename(columns={'text':'Tweets'},inplace=True)
tweets_df4.rename(columns={'text':'Tweets'},inplace=True)

# Give the topic
tweets_df1.loc[:, 'Topics'] = 'earthquakes'
tweets_df2.loc[:, 'Topics'] = 'floods'
tweets_df3.loc[:, 'Topics'] = 'forestfires'
tweets_df4.loc[:, 'Topics'] = 'hurricanes'

# Concatenate
tweets_df = tweets_df1
tweets_df = tweets_df.append(tweets_df2, ignore_index = True)
tweets_df = tweets_df.append(tweets_df3, ignore_index = True)
tweets_df = tweets_df.append(tweets_df4, ignore_index = True)

# Shuffle
tweets_df = tweets_df.sample(frac=1.0, random_state = 100).reset_index(drop=True)

'''

'\n# Former method\n\npath1 = \'/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/earthquakes_eyewitness_crowdflower_2000.tsv\'\npath2 = \'/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/floods_eyewitness_crowdflower_2000.tsv\'\npath3 = \'/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/forestfires_eyewitness_crowdflower_2000.tsv\'\npath4 = \'/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/hurricanes_eyewitness_crowdflower_2000.tsv\'\n\ntweets_df1=pd.read_csv(path1, sep="\t")\ntweets_df2=pd.read_csv(path2, sep="\t")\ntweets_df3=pd.read_csv(path3, sep="\t")\ntweets_df4=pd.read_csv(path4, sep="\t")\n\ntweets_df1 = tweets_df1[[\'text\']]\ntweets_df2 = tweets_df2[[\'text\']]\ntweets_df3 = tweets_df3[[\'text\']]\ntweets_df4 = tweets_df4[[\'text\']]\n\ntweets_df1.rename(columns={\'text\':\'Tweets\'},inplace=True)\ntweets_df2.rename(columns={\'text\':\'Tweets\'},inplace=True)\ntweets_df3.rename(columns={\'text\':\'Tweets

In [ ]:
# Shuffle again
tweets_df = tweets_df.sample(frac=1.0, random_state = 100).reset_index(drop=True)

In [ ]:
# See if the partition is correct
tweets_df.partition.value_counts()

train    6400
test      800
dev       800
Name: partition, dtype: int64

In [ ]:
# Uncomment to see the merged (or unmerged) dataframes
tweets_df

,Tweets,partition,Topics
0,NASA Sees #Tropical #Cyclone #Debbie Approachi...,train,3
1,A small earthquake just happened,train,0
2,'Unprecedented' Wildfires Break Out in Norther...,test,2
3,earthquake in #Oakland just now,train,0
4,I don t know if my heart was beating too fast...,train,0
...,...,...,...
7995,Go Dubs #earthquake #bayarea,dev,0
7996,Guess this explains why my last apartment cons...,train,1
7997,Climatologist explains why 'the conditions are...,train,2
7998,i could've had my braces off today but debbie ...,train,3


In [ ]:
# Make sure you run this one before other methods!

def to_lowercase(text):
    return text.lower()

#testing the function on a single sample for explaination
print(to_lowercase('IN CHINESE WE CALL CAPITALIZATION AS BIG WRITTING, IN GERMAN AS WELL.'))

#converting every row of the column into lower case 
tweets_df.Tweets=tweets_df.Tweets.apply(to_lowercase)

in chinese we call capitalization as big writting, in german as well.


In [ ]:
def standardize_accented_chars(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

#testing the function on a single sample for explaination
print(standardize_accented_chars('sómě words such as résumé, café, prótest, divorcé, coördinate, exposé, latté.'))

#standardizing accented characters for every row
tweets_df.Tweets=tweets_df.Tweets.apply(standardize_accented_chars)

some words such as resume, cafe, protest, divorce, coordinate, expose, latte.


In [ ]:
# Not a method, just to check how many tweets contain urls

def get_number_of_urls(documents):
    print("{:.2f}% of documents contain urls".format(sum
(documents.apply(lambda x:x.find('http'))>0)/len
(documents)*100))

# Passing the 'Tweets' column of the dataframe as the argument
print(get_number_of_urls(tweets_df.Tweets)) 

57.98% of documents contain urls
None


In [ ]:
def remove_url(text):
    return re.sub(r'https?:\S*', '', text)

#testing the function on a single sample for explaination
print(remove_url('using https://www.google.com/ as an example'))

#removing urls from every row
tweets_df.Tweets=tweets_df.Tweets.apply(remove_url)

using  as an example


In [ ]:
def expand_contractions(text):
    expanded_words = [] 
    for word in text.split():
       expanded_words.append(contractions.fix(word)) 
    return ' '.join(expanded_words)

#testing the function on a single sample for explaination
print(expand_contractions("Don't is the same as do not"))

#expanding contractions for every row
tweets_df.Tweets=tweets_df.Tweets.apply(expand_contractions)

Do not is the same as do not


In [ ]:
def remove_mentions_and_tags(text):
    text = re.sub(r'@\S*', '', text)
    return re.sub(r'#\S*', '', text)

#testing the function on a single sample for explaination
print(remove_mentions_and_tags('Some random @abc and #def'))

#removing mentions and tags from every row
tweets_df.Tweets=tweets_df.Tweets.apply(remove_mentions_and_tags)

Some random  and 


In [ ]:
def keep_only_alphabet(text):
    return re.sub(r'[^a-zA-Z]', ' ', text)

#testing the function on a single sample for explaination
print(keep_only_alphabet('Just a bit more $$processing required.Just a bit!!!'))

#for all the rows
tweets_df.Tweets=tweets_df.Tweets.apply(keep_only_alphabet)

Just a bit more   processing required Just a bit   


In [ ]:
def remove_stop_words(text):
  """
  Returns text without stop words
  """
  text = word_tokenize(text)
  word_list = []
  for word in text:
      if word not in stopwords.words('english') and word != 'rt':
          word_list.append(word)

  return ' '.join(word_list)

#testing the function on a single sample for explaination
print(remove_stop_words('rt Test this text to see which are stop words.'))

#removing stop-words and short words from every row
tweets_df.Tweets=tweets_df.Tweets.apply(remove_stop_words)

Test text see stop words .


In [ ]:
def lemmatize(text):
  lemmatizer = WordNetLemmatizer()
  text_str = word_tokenize(text)
  new_words = []

  for word in text_str:
    new_words.append(lemmatizer.lemmatize(word))
  return ' '.join(new_words)

#testing the function on a single sample for explaination
print(lemmatize('apples, bananas and pears are common fruits that are eaten by humans.'))

#Performing lemmatization on every row
tweets_df.Tweets=tweets_df.Tweets.apply(lemmatize)

apple , banana and pear are common fruit that are eaten by human .


In [ ]:
# Delete blank rows to fit OCTIS
tweets_df = tweets_df[tweets_df['Tweets'].str.len()>=1]
tweets_df.reset_index(inplace=True)

# Delete the index column caused by reset_index
del tweets_df['index']

# Delete tweets that only contain 1 or 2 words
for i in range(tweets_df.shape[0]):
    if len(tweets_df['Tweets'][i].split()) < 3:
        tweets_df.drop(index = i, inplace = True)

# (OPTIONAL) Delete tweets using the length of string
#tweets_df = tweets_df[tweets_df['Tweets'].str.len()>=20]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
# Uncomment this to check how the df looks like after running
tweets_df

,Tweets,partition,Topics
0,nasa see approaching landfall nasa aqua,train,3
1,small earthquake happened,train,0
2,unprecedented wildfire break northern southern...,test,2
4,know heart beating fast felt earthquake,train,0
5,cyclone debbie coal mine hopeful reopening soo...,dev,3
...,...,...,...
7983,turn completely flooded school closed car subm...,train,3
7985,guess explains last apartment consistently flo...,train,1
7986,climatologist explains condition primed fire c...,train,2
7987,could brace today debbie damage,train,3


In [ ]:
tweets_df.reset_index(inplace=True)
del tweets_df['index']

Make dataset and vocabulary for OCTIS

In [ ]:
# Create vocabulary.txt

def df_to_vocab(df):
    word_list = []
    for i in range(df.shape[0]):
        text = word_tokenize(df.Tweets[i])
        for word in text:
            if word not in word_list:
                word_list.append(word)
    return word_list

word_list = df_to_vocab(tweets_df)
len(word_list)

9266

In [ ]:
def list_to_vocab(word_list):
    txt = open(f"{save_dir}/vocabulary.txt", 'w')
    for i in range(len(word_list)):
        txt.write(word_list[i])
        txt.write('\r\n')
    txt.close()

list_to_vocab(word_list)

In [ ]:
tweets_df.to_csv(f'{save_dir}/corpus_reduced.tsv',\
                 sep = '\t', index=False, header = None)

In [ ]:
tweets_df.to_csv(f'{save_dir}/corpus_with_header_reduced.tsv',\
                 sep = '\t', index=False)

In [ ]:
# Use this as a check point
#assert False

# OCTIS Initiation

In [ ]:
!pip install octis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 129 kB 14.9 MB/s 
     |████████████████████████████████| 79 kB 10.0 MB/s 
     |████████████████████████████████| 24.1 MB 1.7 MB/s 
     |████████████████████████████████| 22.3 MB 1.2 MB/s 
     |████████████████████████████████| 170 kB 78.0 MB/s 
     |████████████████████████████████| 100 kB 10.8 MB/s 
     |████████████████████████████████| 16.3 MB 60.0 MB/s 
     |████████████████████████████████| 4.4 MB 36.1 MB/s 
     |████████████████████████████████| 1.2 MB 54.7 MB/s 
     |████████████████████████████████| 86 kB 7.0 MB/s 
     |████████████████████████████████| 6.6 MB 46.7 MB/s 
     |████████████████████████████████| 596 kB 58.2 MB/s 
  Created wheel for libsvm: filename=libsvm-3.23.0.4-cp37-cp37m-linux_x86_64.whl size=233369 sha256=2a618fedfdfcf780a1abd31157e1ace9305944e3c4d3b79d3139a44310daa84d
  Stored in directory: /root/.cache/pip/wh

In [ ]:
import octis
from octis.models.LDA import LDA
from octis.models.NMF import NMF
from octis.dataset.dataset import Dataset
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
import numpy as np

In [ ]:
# Save TC and TD
TCTD_line = []
TCTD_all = []

# OCTIS -- LDA with crisis dataset

With our own data preprocessing method

In [ ]:
dataset = Dataset()
dataset.load_custom_dataset_from_folder(save_dir)

In [ ]:
model = LDA(num_topics=NUM_TOPICS, random_state = 98)  # Create model
output_LDA_crisis = model.train_model(dataset) # Train the model

In [ ]:
# See what are in the model output

print(*list(output_LDA_crisis.keys()), sep="\n")

topic-word-matrix
topics
topic-document-matrix
test-topic-document-matrix


In [ ]:
for t in output_LDA_crisis['topics'][:]:
  print(" ".join(t))

wildfire california flood fort mcmurray canada earthquake canadian fire alberta
flood earthquake flash like warning going debbie cyclone right rain
earthquake flood wildfire felt california feel people fire like time
cyclone debbie ex flood queensland school rain flooding wildfire today


In [ ]:
# Initialize metric
npmi = Coherence(texts=dataset.get_corpus(), topk=10, measure='c_v')

# Initialize metric
topic_diversity = TopicDiversity(topk=10)

In [ ]:
# Retrieve metrics score
topic_diversity_score = topic_diversity.score(output_LDA_crisis)
print("Topic diversity: " + str(topic_diversity_score))

npmi_score = npmi.score(output_LDA_crisis)
print("Coherence: " + str(npmi_score))

Topic diversity: 0.675
Coherence: 0.3960299577721983


In [ ]:
TCTD_line.append('Crisis_12')
TCTD_line.append('LDA')
TCTD_line.append(topic_diversity_score)
TCTD_line.append(npmi_score)
TCTD_all.append(TCTD_line)
TCTD_line = []

With OCTIS' data preprocessing method (to do)

In [ ]:
'''
import os
import string
from octis.preprocessing.preprocessing import Preprocessing
os.chdir(os.path.pardir)

# Initialize preprocessing
preprocessor = Preprocessing(vocabulary=None, max_features=None,
                             remove_punctuation=True, punctuation=string.punctuation,
                             lemmatize=True, stopword_list='english',
                             min_chars=1, min_words_docs=0)
# preprocess
dataset = preprocessor.preprocess_dataset(documents_path=r'..\corpus.txt', labels_path=r'..\labels.txt')

# save the preprocessed dataset
dataset.save('hello_dataset')
'''

"\nimport os\nimport string\nfrom octis.preprocessing.preprocessing import Preprocessing\nos.chdir(os.path.pardir)\n\n# Initialize preprocessing\npreprocessor = Preprocessing(vocabulary=None, max_features=None,\n                             remove_punctuation=True, punctuation=string.punctuation,\n                             lemmatize=True, stopword_list='english',\n                             min_chars=1, min_words_docs=0)\n# preprocess\ndataset = preprocessor.preprocess_dataset(documents_path=r'..\\corpus.txt', labels_path=r'..\\labels.txt')\n\n# save the preprocessed dataset\ndataset.save('hello_dataset')\n"

# OCTIS -- LDA with 20News dataset

In [ ]:
# Define dataset
dataset = Dataset()
dataset.fetch_dataset("20NewsGroup")

In [ ]:
# The length of testset
len(dataset.get_corpus()[13862:])

2447

In [ ]:
# Create Model
model = LDA(num_topics=20, alpha=0.1, random_state = 100)

# Train the model
output_LDA_20news = model.train_model(dataset) 

In [ ]:
for t in output_LDA_20news['topics'][:5]:
  print(" ".join(t))

people time make child give religion man day thing word
people government gun state law weapon crime criminal kill make
people make time blue work ticket hand ground police thing
man homosexual make light people good turn leave time gay
key chip encryption make clipper post security people law public


In [ ]:
# Initialize metric
npmi = Coherence(texts=dataset.get_corpus(), topk=10, measure='c_v')

# Initialize metric
topic_diversity = TopicDiversity(topk=10)

In [ ]:
# Retrieve metrics score
topic_diversity_score = topic_diversity.score(output_LDA_20news)
print("Topic diversity: " + str(topic_diversity_score))

npmi_score = npmi.score(output_LDA_20news)
print("Coherence: " + str(npmi_score))

Topic diversity: 0.695
Coherence: 0.5073820330016738


In [ ]:
TCTD_line.append('20_news')
TCTD_line.append('LDA')
TCTD_line.append(topic_diversity_score)
TCTD_line.append(npmi_score)
TCTD_all.append(TCTD_line)
TCTD_line = []

# OCTIS -- NMF with crisis dataset

In [ ]:
dataset = Dataset()
dataset.load_custom_dataset_from_folder(save_dir)

In [ ]:
model = NMF(num_topics=NUM_TOPICS, random_state = 96)  # Create model
output_NMF_crisis = model.train_model(dataset) # Train the model

In [ ]:
for t in output_NMF_crisis['topics'][:]:
  print(" ".join(t))

cyclone debbie ex queensland school tropical australia today via rain
earthquake flood like felt feel home water first anyone going
flood rain flash queensland amp time flooding heavy day warning
wildfire california fort mcmurray fire canada smoke northern canadian alberta


In [ ]:
# Initialize metric
npmi = Coherence(texts=dataset.get_corpus(), topk=10, measure='c_v')

# Initialize metric
topic_diversity = TopicDiversity(topk=10)

In [ ]:
# Retrieve metrics score
topic_diversity_score = topic_diversity.score(output_NMF_crisis)
print("Topic diversity: " + str(topic_diversity_score))

npmi_score = npmi.score(output_NMF_crisis)
print("Coherence: " + str(npmi_score))

Topic diversity: 0.925
Coherence: 0.4740389662093467


In [ ]:
TCTD_line.append('Crisis_12')
TCTD_line.append('NMF')
TCTD_line.append(topic_diversity_score)
TCTD_line.append(npmi_score)
TCTD_all.append(TCTD_line)
TCTD_line = []

# OCTIS -- NMF with 20News dataset

In [ ]:
# Define dataset
dataset = Dataset()
dataset.fetch_dataset("20NewsGroup")

In [ ]:
# Create Model
model = NMF(num_topics=20, random_state = 100)

# Train the model
output_NMF_20news = model.train_model(dataset) 

In [ ]:
for t in output_NMF_20news['topics'][:5]:
  print(" ".join(t))

server include base support send widget subject motif mail source
system user list information internet application post run window email
system key question atheist post argument exist belief true religion
graphic image mail send package format object datum model include
government turkish russian administration official support fund american war food


In [ ]:
# Initialize metric
npmi = Coherence(texts=dataset.get_corpus(), topk=10, measure='c_v')

# Initialize metric
topic_diversity = TopicDiversity(topk=10)

In [ ]:
# Retrieve metrics score
topic_diversity_score = topic_diversity.score(output_NMF_20news)
print("Topic diversity: " + str(topic_diversity_score))

npmi_score = npmi.score(output_NMF_20news)
print("Coherence: " + str(npmi_score))

Topic diversity: 0.695
Coherence: 0.5978809563977465


In [ ]:
TCTD_line.append('20_news')
TCTD_line.append('NMF')
TCTD_line.append(topic_diversity_score)
TCTD_line.append(npmi_score)
TCTD_all.append(TCTD_line)
TCTD_line = []

# Evaluation

In [ ]:
# TC and TD
col_names_TCTD = ("Dataset", "algorithm", "Topic Coherence", "Topic Diversity")
TCTD_df = pd.DataFrame(TCTD_all, columns=col_names_TCTD)

TCTD_df

,Dataset,algorithm,Topic Coherence,Topic Diversity
0,Crisis_12,LDA,0.675,0.396030
1,20_news,LDA,0.695,0.507382
2,Crisis_12,NMF,0.925,0.474039
3,20_news,NMF,0.695,0.597881


In [ ]:
a = dataset._Dataset__corpus[0]

In [ ]:
len(dataset.get_labels())

16309

In [ ]:
# Match the number of labels to topics
# Adjust according to the output in the block below

# Dataset_12
in_labels = {0:'earthquake',
             1:'floods',
             2:'forestfires',
             3:'hurricanes'}

out_labels = {0:'hurricanes',
              1:'earthquake',
              2:'floods',
              3:'forestfires'}

# 20 news dataset (Not accurate, we don't use this for now)
#labels_20news = {}
#for i in range(20):
#    labels_20news[i] = dataset._Dataset__metadata['labels'][i]

In [ ]:
# Make 20 news test dataset
# testset_20news: The texts
# testset_20news_labels: The labels
dataset = Dataset()
dataset.fetch_dataset("20NewsGroup")

testset_20news_orig = dataset.get_corpus()[13862:]
testset_20news = []
for i in range(len(testset_20news_orig)):
    full = ' '.join(s for s in testset_20news_orig[i])
    testset_20news.append(full)
testset_20news_labels = dataset.get_labels()[13862:]

In [ ]:
for t in output_NMF_crisis['topics'][:]:
  print(" ".join(t))

cyclone debbie ex queensland school tropical australia today via rain
earthquake flood like felt feel home water first anyone going
flood rain flash queensland amp time flooding heavy day warning
wildfire california fort mcmurray fire canada smoke northern canadian alberta


In [ ]:
# Doc-Topic Output (Only test set here)
# Why do we need "run_id"
# 4 changes to make in total when changing the algorithm (and the corresponding dataset)
# 1 change to make when changing the dataset (by commenting)

col_names_DT = ("Document", "Real Label", "Assigned Topic Num", "Assignment Score")
result_DT = []

# Dataset 1
#df = pd.read_csv('/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_1/corpus_with_header.tsv', delimiter='\t')

# Dataset 12
df = pd.read_csv('/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12/corpus_with_header.tsv', delimiter='\t')

# Crisis dataset
df = df.loc[df['partition'] == 'test']
output_max = np.max(output_LDA_crisis['test-topic-document-matrix'], axis = 0)
output_index = np.argmax(output_LDA_crisis['test-topic-document-matrix'], axis = 0)
length = len(output_max)

# 20 news dataset (didn't perform really well here, so we don't use it for now)
#length = len(testset_20news)
#output_max = np.max(output_NMF_20news['test-topic-document-matrix'], axis = 0)
#output_index = np.argmax(output_NMF_20news['test-topic-document-matrix'], axis = 0)

for i in range(length):
    result_DT_line = []

    # Crisis dataset
    result_DT_line.append(df[i:i+1].Tweets.values[0])
    result_DT_line.append(in_labels[df[i:i+1].Topics.values[0]])

    # 20 news dataset
    #result_DT_line.append(testset_20news[i])
    #result_DT_line.append(testset_20news_labels[i])

    result_DT_line.append(output_index[i])
    result_DT_line.append(output_max[i])

    result_DT.append(result_DT_line)

Doc_Topic_df = pd.DataFrame(result_DT, columns=col_names_DT)

Doc_Topic_df

,Document,Real Label,Assigned Topic Num,Assignment Score
0,unprecedented wildfire break northern southern...,forestfires,0,0.700985
1,heatwaves flood warning going place thought to...,floods,1,0.737232
2,wildfire devastate northern southern,forestfires,0,0.848227
3,uploaded month patreon little earlier case cyc...,hurricanes,1,0.662079
4,morrison still extreme wildfire behaviour,forestfires,0,0.723109
...,...,...,...,...
779,see feel like escape hellish bbc vine bbcworld,forestfires,2,0.595913
780,earthquake japan shortened day planet earth mi...,earthquake,2,0.618198
781,grocery price rise across australia cyclone de...,hurricanes,3,0.928491
782,bbcnews pm california wildfire serious usually...,forestfires,0,0.854030


In [ ]:
# Method from Berk's work, 3q!
# Define length according to the dataset
length = 4

for i in range(length):
    print(f'Topic {i}:')
    print(Doc_Topic_df.query(f'`Assigned Topic Num` == {i}') ['Real Label'].value_counts())
    print('-'*32)

Topic 0:
forestfires    140
earthquake      51
floods           7
hurricanes       3
Name: Real Label, dtype: int64
--------------------------------
Topic 1:
floods         143
earthquake      27
hurricanes      12
forestfires      6
Name: Real Label, dtype: int64
--------------------------------
Topic 2:
earthquake     64
forestfires    43
floods         32
hurricanes     13
Name: Real Label, dtype: int64
--------------------------------
Topic 3:
hurricanes     172
earthquake      46
floods          17
forestfires      8
Name: Real Label, dtype: int64
--------------------------------


In [ ]:
# Topic-Word Output
# 1 change to make in total when changing the algorithm (and the corresponding dataset)
# 1 change to make when changing the dataset (by commenting)
# Remember to change NUM_TOPICS, it's also possible to change it here
NUM_TOPICS = 20

col_names_TW = ("num_given_topics", "topic_num", "topic_words")
result_TW = []

for i in range(NUM_TOPICS):
    result_TW_line = []

    result_TW_line.append(NUM_TOPICS)
    result_TW_line.append(i)
    result_TW_line.append(output_LDA_20news['topics'][i])
    result_TW.append(result_TW_line)

Topic_word_df = pd.DataFrame(result_TW, columns=col_names_TW)
Topic_word_df

,num_given_topics,topic_num,topic_words
0,20,0,"[people, time, make, child, give, religion, ma..."
1,20,1,"[people, government, gun, state, law, weapon, ..."
2,20,2,"[people, make, time, blue, work, ticket, hand,..."
3,20,3,"[man, homosexual, make, light, people, good, t..."
4,20,4,"[key, chip, encryption, make, clipper, post, s..."
5,20,5,"[armenian, space, turkish, launch, year, russi..."
6,20,6,"[color, driver, bit, mode, card, run, work, di..."
7,20,7,"[page, title, text, include, information, dept..."
8,20,8,"[file, image, mail, program, information, incl..."
9,20,9,"[win, year, gun, make, team, rate, lose, back,..."


In [ ]:
word_list.index('window')

614

In [ ]:
output_LDA_20news['topics']

[['people',
  'time',
  'make',
  'child',
  'give',
  'religion',
  'man',
  'day',
  'thing',
  'word'],
 ['people',
  'government',
  'gun',
  'state',
  'law',
  'weapon',
  'crime',
  'criminal',
  'kill',
  'make'],
 ['people',
  'make',
  'time',
  'blue',
  'work',
  'ticket',
  'hand',
  'ground',
  'police',
  'thing'],
 ['man',
  'homosexual',
  'make',
  'light',
  'people',
  'good',
  'turn',
  'leave',
  'time',
  'gay'],
 ['key',
  'chip',
  'encryption',
  'make',
  'clipper',
  'post',
  'security',
  'people',
  'law',
  'public'],
 ['armenian',
  'space',
  'turkish',
  'launch',
  'year',
  'russian',
  'genocide',
  'report',
  'greek',
  'satellite'],
 ['color',
  'driver',
  'bit',
  'mode',
  'card',
  'run',
  'work',
  'display',
  'window',
  'set'],
 ['page',
  'title',
  'text',
  'include',
  'information',
  'depth',
  'character',
  'program',
  'copy',
  'year'],
 ['file',
  'image',
  'mail',
  'program',
  'information',
  'include',
  'send',
  'sof

In [ ]:
len(word_list)

9266

Test (Delete when finished)

In [ ]:
dataset = Dataset()
dataset.load_custom_dataset_from_folder(save_dir)
# Create Model
model = NMF(num_topics=4)

# Train the model
output = model.train_model(dataset) 

Note the 4 outputs:


*   *topics*: the list of word topics
*   *topic-word-matrix*: the distribution of the words of the vocabulary for each topic (dimensions: |num topics| x |vocabulary|)
*   *topic-document-matrix*: the distribution of the topics for each document of the training set (dimensions: |num topics| x |training documents|)
*   *test-topic-document-matrix*: the distribution of the topics for each document of the testing set (dimensions: |num topics| x |test documents|)


In [ ]:
print(*list(output.keys()), sep="\n")

topic-word-matrix
topics
topic-document-matrix
test-topic-document-matrix


In [ ]:
import numpy as np
#output_LDA_20news
output_LDA_20news['test-topic-document-matrix']

array([[0.        , 0.        , 0.        , ..., 0.4209156 , 0.01250645,
        0.        ],
       [0.        , 0.46702215, 0.        , ..., 0.22719279, 0.0125052 ,
        0.44856879],
       [0.        , 0.        , 0.        , ..., 0.        , 0.32633793,
        0.31227356],
       ...,
       [0.        , 0.18373396, 0.        , ..., 0.        , 0.0125042 ,
        0.        ],
       [0.21632406, 0.        , 0.        , ..., 0.        , 0.44857687,
        0.        ],
       [0.0658233 , 0.        , 0.        , ..., 0.2987439 , 0.01250443,
        0.        ]])

In [ ]:
output['test-topic-document-matrix']
output_max = np.max(output['test-topic-document-matrix'], axis = 0)
output_index = np.argmax(output['test-topic-document-matrix'], axis = 0)

output_index

array([2, 3, 2, 0, 2, 1, 2, 2, 2, 1, 1, 0, 3, 1, 0, 3, 1, 2, 1, 3, 2, 3,
       2, 2, 1, 3, 2, 1, 1, 2, 3, 0, 3, 3, 3, 3, 0, 1, 0, 1, 0, 0, 1, 3,
       0, 1, 3, 2, 2, 0, 3, 2, 1, 0, 0, 2, 0, 2, 3, 3, 1, 3, 1, 1, 3, 2,
       2, 1, 2, 2, 1, 2, 2, 1, 0, 2, 0, 1, 1, 1, 0, 3, 0, 3, 0, 3, 3, 3,
       3, 0, 0, 1, 3, 3, 1, 1, 0, 2, 0, 3, 3, 3, 1, 1, 1, 1, 1, 1, 2, 3,
       3, 0, 2, 0, 2, 0, 0, 0, 3, 3, 1, 2, 3, 1, 0, 1, 2, 3, 1, 0, 0, 0,
       1, 0, 2, 2, 3, 3, 2, 0, 2, 0, 1, 2, 2, 0, 3, 3, 1, 0, 2, 3, 0, 3,
       3, 3, 3, 3, 1, 1, 2, 2, 1, 1, 3, 3, 2, 3, 0, 3, 0, 3, 3, 2, 3, 1,
       0, 2, 0, 0, 2, 1, 3, 1, 0, 0, 1, 2, 3, 0, 0, 2, 0, 2, 0, 3, 3, 0,
       3, 1, 3, 0, 2, 2, 1, 3, 3, 3, 1, 0, 0, 0, 0, 2, 0, 1, 1, 3, 0, 1,
       3, 1, 3, 0, 2, 1, 0, 1, 0, 3, 1, 2, 1, 0, 0, 1, 2, 3, 2, 3, 0, 3,
       1, 0, 0, 0, 3, 3, 1, 3, 0, 1, 1, 2, 2, 0, 2, 1, 1, 1, 1, 2, 3, 2,
       3, 1, 0, 0, 3, 0, 0, 2, 1, 3, 3, 0, 1, 3, 3, 0, 3, 0, 1, 3, 2, 3,
       3, 0, 3, 1, 1, 3, 2, 1, 0, 2, 1, 3, 2, 0, 0,

In [ ]:
output

{'test-topic-document-matrix': array([[0.01993886, 0.        , 0.        , ..., 0.98156393, 0.03036368,
         0.        ],
        [0.        , 0.0677283 , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.98006114, 0.06097251, 1.        , ..., 0.        , 0.96963632,
         0.1148065 ],
        [0.        , 0.87129919, 0.        , ..., 0.01843607, 0.        ,
         0.8851935 ]]),
 'topic-document-matrix': array([[0.26720211, 0.        , 0.        , ..., 0.14275732, 0.90111454,
         0.09516059],
        [0.01187323, 1.        , 1.        , ..., 0.21519976, 0.        ,
         0.00552508],
        [0.60932501, 0.        , 0.        , ..., 0.64204292, 0.        ,
         0.84844574],
        [0.11159965, 0.        , 0.        , ..., 0.        , 0.09888546,
         0.0508686 ]]),
 'topic-word-matrix': array([[1.31396175e-04, 0.00000000e+00, 1.51972218e-03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [9.64183888e-06, 0.00000